# (PART3) 흥미로운 GAN 기법

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pandas, numpy, random
import matplotlib.pyplot as plt

## (CHAPTER9) 합성곱 GAN  

### 메모리 소비  
* (크게 영양가 없는 내용으로 pass 😅)  

### 지역화된 이미지 특성  
* <span style='color:red'><b>머신러닝의 황금 규칙은 가능한 모든 지식을 총 동원해서 문제를 푸는 것임.</b></span>  
* <span style='color:red'><b>도메인 지식을 활용해 쓸모없는 방법을 거르고, 좋은것들만 조합을 만들어서 적용하는것이 빠름.</b></span>  
* 이미지의 유의미한 특성은 지역적인 특성(localized)임  (ex) 눈과 코는 근처에 위치함.  
* 이러한 직관을 사용해, 신경망 분류를 디자인하면 더욱 효과적임.  

### 합성곱 필터  
* (크게 영양가 없는 내용으로 pass 😅)  

### 커널 가중치 학습하기  
* (크게 영양가 없는 내용으로 pass 😅)  

### 특성의 계층구조  
* 특성 맵에 다시 합성곱 커널의 레이어를 추가하면 고수준의 특성을 얻을 수 있음.  
* low level -> middle level -> high level로 계층적인 분석을 거치도록 구조 설계가 가능함.  

### MNIST CNN

* <code>LayerNorm()</code>대신 <code>BatchNorm()</code>으로 변경 <span style="color:green"># 이 부분을 왜 변경했는가?<span>  


In [2]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape, # 튜플로 바꿈(왜냐하면 *을 이용해서 flatten하기 위해)
        
    def forward(self, x:torch.Tensor):
        return x.view(*self.shape)

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 200),
            # nn.Sigmoid(),
            nn.LeakyReLU(0.02), # 2️⃣
            nn.LayerNorm(200), # 3️⃣
            nn.Linear(200, 1),
            nn.Sigmoid()
        )
        
        # self.loss_function = nn.MSELoss()
        self.loss_function = nn.BCELoss() # 1️⃣
        # self.optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-4) # 4️⃣
        self.counter = 0 # count epoch
        self.progress = [] # for appending loss
    

    def train(self, inputs, targets):
        """params update와 logging까지 같이 수행하는 함수

        Args:
            inputs (torch.Tensor): 데이터
            targets (torch.Tensor): 라벨
        """
        outputs = self.forward(inputs) # forward
        loss:torch.Tensor = self.loss_function(outputs, targets) # loss
        
        # appending loss
        self.counter += 1
        if  (self.counter % 10 == 0):
            self.progress.append(loss.item())
        # print info
        if (self.counter % 10000 == 0):
            print(f"counter = {self.counter}")
            

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def plot_progress(self):
        df = pandas.DataFrame(self.progress, columns=["loss"])
        df.plot(
            ylim=(0), # 범위 제한 삭제
            figsize=(8, 4), alpha=0.1, marker=".", grid=True, yticks=(0, 0.25, 0.5, 1.0, 5.0),
        )
    
    def forward(self, x):
        x = self.model(x)
        return x

### CelebA CNN  

In [ ]:
def crop_centre(img, new_width, new_height):
    return None

* 지역화된 패치가 하나씩 붙어있는 모양새임.    
* 이것으로 가늠컨데, 의도적으로 얼굴 특정 부분에 집중하는 것은 장점도 있으나 단점도 존재함.  

### 각자 실험해보기

* <code>nn.LeakyReLU()</code>대신 <code>nn.GELU()</code>를 사용

### 핵심 정리
* CNN은 로컬 정보를 통해 전체 이미지의 특성을 찾을 수 있다는 점을 활용함.  
* CNN의 특성 탐색은 low, middle, high로 이루어져 있음.  
* CNN을 사용한 Generator는 얼굴 요소들의 조화가 깨질 수 있음.  
* 왜냐하면, 지역화된 정보만 학습하고 전체적인 관계는 학습하지 않게 구현했기 때문.  

## (CHAPTER10) 조건부 GAN

### 조건부 GAN 구조
* 생성기에 임의의 시드 + 어떤 이미지를 원하는지 입력을 넣어줘야 함.  
* 클레스 레이블과 이미지 사이의 관계를 추가학습할 필요가 있음.  
* 생성기와 판별기 모두 이미지 데이터와 클레스 레이블을 추가로 받음.  

### 판별기  
* 이미지 텐서와 원핫으로 변환된 레이블 텐서를 단순 결합(concat)  

### 생성기  

...

### 훈련 반복문  
* random_label 변수를 사용해서 판별기, 생성기에 같은 레이블 텐서를 주입함.  

### 차트 그리기

...

### 조건부 GAN 결과 확인하기

...

### 핵심 정리  
* 일반적인 GAN과 달리, 조건부 GAN은 원하는 클래스의 데이터 생성이 가능함.  
* 조건부 GAN은 훈련시 판별기에 이미지를 보강해서 전달, 생성기에는 클래스 레이블을 통해 시드가 투입.  
* <span style='color:red'>조건부 GAN은 레이블 정보를 받지 않는 일반 GAN보다 이미지 품질이 더 좋음.</span>